In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt, patches
from datetime import datetime, timedelta
from tqdm import tqdm
from scipy import interpolate

import process_NEMO_obs_matching as tools
from importlib import reload

%matplotlib inline

In [ ]:
# Extra code
# Define horizontal slices
slc = {'y': jslc, 'x': islc}

# Expand slices for unstaggering
if gridtype in ['u', 'v']:
    coord = dims_mapping[gridtype]
    slc[coord] = slice(slc[coord].start, slc[coord].stop + 1)

# Slice and interpolate ssh
ssh = ssh.isel(slc).interp(time_counter=time)

# Extrapolate over mask edges
slc = (0,) + tuple(slc.values())
ssh = ssh.where(gridvars['mask'][slc]).interpolate_na(dim=coord, **kwargs)

# Unstagger to velocity grid
if gridtype in ['u', 'v']:
    ssh = ssh.interp({coord: np.arange(0.5, w)})

# Depth array
depths = gridvars['depth'] * np.expand_dims(1 + ssh / gridvars['H'], axis=0)

# Extrapolate depth NaNs
depths_nan = []
da_nan = xr.DataArray(np.broadcast_to(np.nan, (w, w)), name='gdept_0', dims=('y', 'x'))
for k, depth in enumerate(depths):
    try:
        depths_nan.append(depth.interpolate_na('x', **kwargs))
    except:
        try:
            depths_nan.append(depth.interpolate_na('y', **kwargs))
        except:
            depths_nan.append(da_nan)
depths_nan = xr.concat(depths_nan, dim='depth')